In [0]:
PROJECT_PATH = '../python'
ISOCHRONES_PATH = '/datascope/subaru/data/isochrones/mist/import/MIST_v1.2_vvcrit0.0_HSC.h5'

MAG1, MAG2, MAG3 = 'hsc_g', 'hsc_i', 'hsc_i'

In [0]:
%matplotlib inline

In [0]:
import os, sys
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import AutoMinorLocator, MultipleLocator

In [0]:
plt.rc('font', size=7)

In [0]:
# Disable GPU access for TensorFlow, use CPU instead.
# Use this option if using shared GPUs on big servers to prevent holding the GPU memory
# when notebooks are left running for a longer time. Do not use this option when
# high performance execution is required.
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [0]:
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

In [0]:
tf.config.list_physical_devices('CPU')

In [0]:
tf.config.list_physical_devices('GPU') 

In [0]:
for p in reversed(PROJECT_PATH.split(':')):
    sys.path.insert(0, p)

# Evaluate isochrone grid

## Load grid from HDF5

In [0]:
from pfs.ga.isochrones import IsoGrid

In [0]:
iso = IsoGrid()
iso.load(os.path.expandvars(ISOCHRONES_PATH))

In [0]:
iso.axes.keys()

In [0]:
for k in iso.values.keys():
    print(k, 
          tf.math.count_nonzero(tf.math.is_inf(iso.values[k])).numpy(),
          tf.math.count_nonzero(tf.math.is_nan(iso.values[k])).numpy())

## Atmospheric parameter ranges

In [0]:
def get_range(k):
    mask = ~tf.math.is_inf(iso.values[k])
    mn = tf.math.reduce_min(tf.where(mask, iso.values[k], 1e10))
    mx = tf.math.reduce_max(tf.where(mask, iso.values[k], 0))
    return mn.numpy(), mx.numpy()
    
log_t_eff_min, log_t_eff_max = get_range('log_T_eff')
print('T_eff', 10 ** log_t_eff_min, 10 ** log_t_eff_max)
print('log_g', get_range('log_g'))
print('[Fe/H]', iso.Fe_H[0].numpy(), iso.Fe_H[-1].numpy())

## Plot the isochrones

In [0]:
iso.Fe_H

In [0]:
iso.log_t

In [0]:
iso.EEP

In [0]:
f, ax = plt.subplots(1, 1, figsize=(6, 6), dpi=120)

X = (iso.values[MAG1] - iso.values[MAG2]).numpy().flatten()
Y = iso.values[MAG3].numpy().flatten()

ax.plot(X, Y, 'sk', ms=0.1, alpha=0.1, rasterized=True)
ax.invert_yaxis()

## Plot a few isochrones

In [0]:
iso.values[MAG1].shape

In [0]:
iso.log_t

In [0]:
Fe_H = tf.convert_to_tensor(-0.5, dtype=tf.float64)
print('Fe_H', iso._ip._find_nearby(iso.Fe_H, Fe_H))

log_t = tf.convert_to_tensor(np.log10(1e9), dtype=tf.float64)
print('log_t', iso._ip._find_nearby(iso.log_t, log_t))

EEP = tf.convert_to_tensor([202, 605], dtype=tf.float64)  # ZAMS, ToRGB
print('EEP', iso._ip._find_nearby(iso.EEP, EEP))

In [0]:
iso.axes.keys()

In [0]:
iso.values.keys()

In [0]:
f, axs = plt.subplots(2, 2, figsize=(3.4, 4.5), facecolor='w', dpi=240, sharey=False, squeeze=False)

s = np.s_[4, 102, 202:606]      # -2.0, 13Gyr
log_T_eff = iso.values['log_T_eff'][s]
log_g = iso.values['log_g'][s]
axs[0, 0].plot(10**log_T_eff, log_g, '-b', lw=0.5)

m1 = iso.values[MAG1][s]
m2 = iso.values[MAG2][s]
axs[1, 0].plot(m1 - m2, m1, 'b-', lw=0.5)

s = np.s_[7, 102, 202:606]      # -1.5, 13Gyr
log_T_eff = iso.values['log_T_eff'][s]
log_g = iso.values['log_g'][s]
axs[0, 0].plot(10**log_T_eff, log_g, '-b', lw=0.5)

m1 = iso.values[MAG1][s]
m2 = iso.values[MAG2][s]
axs[1, 0].plot(m1 - m2, m1, 'b-', lw=0.5)

for p in [(4000, 5.0), (4750, 4.5), (5500, 4.5), (4000, 0.5), (4750, 2.0), (5500, 3.5)]:
    axs[0, 0].plot(*p, '.r')

for T_eff in [4000, 4750, 5500]:
    axs[0, 0].axvline(T_eff, ls='--', c='k', lw=0.5)

# axs[0, 0].text(0, 1, '\n [Fe/H] = $-2.0, -1.5$',
#                va='top', ha='left', transform=axs[0, 0].transAxes)
axs[0, 0].set_title('[Fe/H] = $-2.0, -1.5$\nage = 13 Gyr', fontsize=7)

###

s = np.s_[9, 86, 202:606]      # -0.7, 2Gyr
log_T_eff = iso.values['log_T_eff'][s]
log_g = iso.values['log_g'][s]
axs[0, 1].plot(10**log_T_eff, log_g, '-b', lw=0.5)

m1 = iso.values[MAG1][s]
m2 = iso.values[MAG2][s]
axs[1, 1].plot(m1 - m2, m1, 'b-', lw=0.5)

s = np.s_[11, 86, 202:606]      # -0.5, 2Gyr
log_T_eff = iso.values['log_T_eff'][s]
log_g = iso.values['log_g'][s]
axs[0, 1].plot(10**log_T_eff, log_g, '-b', lw=0.5)

m1 = iso.values[MAG1][s]
m2 = iso.values[MAG2][s]
axs[1, 1].plot(m1 - m2, m1, 'b-', lw=0.5)

for p in [(3500, 5.0), (4500, 4.5), (5500, 4.5), (3500, 0.5), (4500, 2.0), (5500, 3.5)]:
    axs[0, 1].plot(*p, '.r')

for T_eff in [3500, 4500, 5500]:
    axs[0, 1].axvline(T_eff, ls='--', c='k', lw=0.5)

axs[0, 1].set_title('[Fe/H] = $-0.7, -0.5$\nage = 2 Gyr', fontsize=7)

###

for ax in axs[0, :].ravel():
    ax.set_ylim(0.1, 5.9)
    ax.set_xlabel(r'$T_\mathrm{eff}$ [K]')
    ax.invert_xaxis()
    ax.invert_yaxis()

for ax in axs[0, :-1].ravel():
    ax.set_ylabel(r'$\log g$')

for ax in axs[0, 1:].ravel():
    ax.yaxis.set_ticklabels([])

###

for ax in axs[1, :].ravel():
    ax.set_xlabel(r'HSC g - HSC i')
    ax.set_xlim(-0.5, 3.5)
    ax.set_ylim(-2.5, 14.5)
    
    ax.invert_yaxis()

for ax in axs[1, :-1].ravel():
    ax.set_ylabel(r'HSC i')

for ax in axs[1, 1:].ravel():
    ax.yaxis.set_ticklabels([])

for ax in axs.ravel():
    ax.tick_params(which='major', left=True, top=True, bottom=True, right=True)
    ax.tick_params(which='minor', left=False, top=False, bottom=False, right=False)
    ax.tick_params(which='major', axis="both", direction="in")


#     ax.xaxis.set_minor_locator(AutoMinorLocator(4))
#     ax.yaxis.set_minor_locator(AutoMinorLocator(4))

#     #ax.grid(True)
#     #ax.grid(which='minor', axis='both', alpha=0.15)

f.tight_layout()

In [0]:
# metallicity, age, EEP
# s = np.s_[4, 102, 202:606]      # -2.0, 13Gyr
# s = np.s_[6, 102, 202:606]      # -1.5, 13Gyr
s = np.s_[9, 86, 202:606]      # -0.5, 2Gyr
# s = np.s_[12, 80, 202:606]      # 0.0, 1Gyr
# s = np.s_[12, 86, 202:606]      # 0.0, 2Gyr

f, axs = plt.subplots(1, 2, figsize=(5, 2.5), facecolor='w', dpi=240)

m1 = iso.values[MAG1][s]
m2 = iso.values[MAG2][s]

axs[0].plot(m1 - m2, m1, '-')

axs[0].set_xlabel(f'${MAG1} - {MAG2}$')
axs[0].set_ylabel(f'${MAG3}$')
#axs[0].set_ylim(14, -5)
axs[0].invert_yaxis()

axs[0].xaxis.set_minor_locator(AutoMinorLocator(5))
axs[0].yaxis.set_minor_locator(AutoMinorLocator(5))

axs[0].grid(True)
axs[0].grid(which='minor', axis='both', alpha=0.15)

###

log_T_eff = iso.values['log_T_eff'][s]
log_g = iso.values['log_g'][s]

axs[1].plot(10**log_T_eff, log_g, '-')

# axs[1].axvline(4000, c='r')
# axs[1].axvline(4750, c='r')
# axs[1].axvline(5500, c='r')

axs[1].axvline(3500, c='r')
axs[1].axvline(4500, c='r')
axs[1].axvline(5500, c='r')

# axs[1].axvline(3500, c='r')
# axs[1].axvline(4750, c='r')
# axs[1].axvline(5250, c='r')

axs[1].invert_xaxis()
axs[1].invert_yaxis()

axs[1].xaxis.set_minor_locator(AutoMinorLocator(4))
axs[1].yaxis.set_minor_locator(AutoMinorLocator(4))

axs[1].grid(True)
axs[1].grid(which='minor', axis='both', alpha=0.15)

## Plot parameter coverage

In [0]:
value = iso.values[MAG1].numpy()
index = np.where(~np.isnan(value) & ~np.isinf(value))
index

In [0]:
iso.Fe_H

In [0]:
plt.figure(figsize=(6, 6), dpi=96, facecolor='w')
plt.plot(iso.Fe_H.numpy()[index[0]], iso.log_t.numpy()[index[1]], '.', alpha=0.01)
plt.xlabel('[Fe/H]')
plt.ylabel('log age (Gyr)')

In [0]:
for fe_h in range(iso.Fe_H.shape[0]):
    filt = index[0] == fe_h

    plt.figure(figsize=(6, 6), dpi=96, facecolor='w')
    plt.plot(iso.log_t.numpy()[index[1][filt]], iso.EEP.numpy()[index[2][filt]], '.')
    plt.xlabel('log age (Gyr)')
    plt.ylabel('EEP')
    plt.title('[Fe/H] = {}'.format(iso.Fe_H[fe_h]))